In [3]:
import pandas as pd
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import DAG, IIDSimulation
from castle.algorithms import PC
from castle.common.priori_knowledge import PrioriKnowledge

import networkx as nx
from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Causal graph
causal_graph = nx.DiGraph([('d', 'messageReceived'),
                           ('d', 'graft'),
                           ('d', 'prune'),
                           ('d', 'iwant'),
                           ('dlo', 'graft'),
                           ('dhi', 'prune'),
                           ('dscore', 'graft'),
                           ('dscore', 'prune'),
                           ('dlazy', 'iwant'),
                           ('dlazy', 'ihave'),
                           ('dout', 'graft'),
                           ('dout', 'prune'),
                           ('gossipFactor', 'iwant'),
                           ('gossipFactor', 'ihave'),
                           ('interval', 'graft'),
                           ('interval', 'prune'),
                           ('interval', 'iwant'),
                           ('interval', 'ihave'),
                           ('topology', 'graft'),
                           ('topology', 'prune'),
                           ('topology', 'propagationTime'),
                           ('topology', 'messageReceived'),
                           ('prune', 'graft'),
                           ('prune', 'propagationTime'),
                           ('messageReceived', 'messageOverhead'),
                           ('messageOverhead', 'bandwidth'),
                           ('iwant', 'bandwidth'),
                           ('ihave', 'bandwidth'),
                           ('messageReceived', 'bandwidth')
                         ])

In [8]:
#From digraph to adjancency matrix

true_causal_matrix = nx.adjacency_matrix(causal_graph)

print(true_causal_matrix)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (1, 15)	1
  (1, 16)	1
  (3, 2)	1
  (3, 14)	1
  (4, 16)	1
  (5, 2)	1
  (6, 3)	1
  (7, 2)	1
  (7, 3)	1
  (8, 4)	1
  (8, 9)	1
  (9, 16)	1
  (10, 2)	1
  (10, 3)	1
  (11, 4)	1
  (11, 9)	1
  (12, 2)	1
  (12, 3)	1
  (12, 4)	1
  (12, 9)	1
  (13, 1)	1
  (13, 2)	1
  (13, 3)	1
  (13, 14)	1
  (15, 16)	1
